Import Panda und SQLalchemy

In [ ]:
import pandas as pd
import sqlalchemy as sql

csv-File importieren.

In [ ]:
sh_2022 = pd.read_csv("data/SH_2022.csv", delimiter= ";", encoding= 'windows-1252')
sh_2022.head(20)
sh_2022.info()

Drop die ersten zwei Columns, die nicht gebraucht werden

In [ ]:
sh_2022 = sh_2022.drop(['Ist 2020 T€','Soll 2021 T€'], axis=1)

Columns names anpassen, Sonderzeichen entfernen, in tausend Euro entfernen

In [ ]:
sh_2022.columns = sh_2022.columns.str.replace(" ", "_")
sh_2022.columns = sh_2022.columns.str.lower()
sh_2022.rename(columns={"ansatz_2022_t€":"ansatz_2022_euro"}, inplace=True)
sh_2022


Values ins passende Format setzten und leading-0 einfügen, wo sie automatisch entfernt wurde

In [ ]:
sh_2022['ep'] = sh_2022['ep'].astype(str)
sh_2022['kapitel'] = sh_2022['kapitel'].astype(str)
sh_2022['titel'] = sh_2022['titel'].astype(str)
sh_2022['titel'] = sh_2022['titel'].apply(lambda x:x.zfill(5))
sh_2022['ep'] = sh_2022['ep'].apply(lambda x:x.zfill(2))
sh_2022['kapitel'] = sh_2022['kapitel'].str[-2:]
sh_2022['kapitel'] = sh_2022['ep'].apply(lambda x:x.zfill(3))
sh_2022

Bei den Zahlen . und , Systematik angepasst und *1000 genommen und von string zu float und dann INT umgewandelt.

In [ ]:
sh_2022.ansatz_2022_euro = sh_2022['ansatz_2022_euro'].str.replace('.', '').str.replace(",", ".").astype(float)
sh_2022.ansatz_2022_euro = sh_2022['ansatz_2022_euro'].apply(lambda x:int(x*1000))

Titel in gruppe und counter gesplittet, gruppe und counter eingefügt

In [ ]:
sh_2022["gruppe"] = sh_2022["titel"].str[:3]
sh_2022["counter"] = sh_2022["titel"].str[3:]
sh_2022.info()

jahr und state_id eingefügt

In [ ]:
sh_2022.insert(8, "jahr", 2022)
sh_2022.insert(9, "state_id", 15)
sh_2022.info()

titel gedropt, da redundant

In [ ]:
sh_2022 = sh_2022.drop("titel", axis=1)
sh_2022.head(20)

anhand der gruppe einnahmen und ausgaben in den Zahlen definiert

In [ ]:
sh_2022['ansatz_2022_euro'] =sh_2022[['ansatz_2022_euro', 'gruppe']].apply(lambda x:-x[0] if int(x[1]) > 400 else x[0], axis=1)


mgtg gedropt, hat keinen mehrwert und viele nan

In [ ]:
sh_2022 = sh_2022.drop(['mgtg'], axis=1)


einmal angucken, da schön

In [ ]:
sh_2022.head()

Importieren unserer Funktionen zum Uploaden bei SQL

In [ ]:
from sql_functions import get_sql_config
#from sql_functions import push_to_database
from sql_functions import get_engine
import psycopg2

Wir bereiten alles für den Push in die Database vor.

In [ ]:
get_sql_config()
engine = get_engine()



Nun können wir pushen.

In [ ]:
#push_to_database(sh_2022, "budget_text_sh", engine , "capstone_public_budgeting")

Wir ergänzen das nun hinsichtlich der Codierungstabellen von SH: Einlesen und hochladen.

In [ ]:
codierung_sh_2022_EP = pd.read_csv("data/Codierung_SH - EP_Bezeichnung_2022.csv")
codierung_sh_2021_EP = pd.read_csv("data/Codierung_SH - EP_Bezeichnung_2021.csv")
codierung_sh_2020_EP = pd.read_csv("data/Codierung_SH - EP_Bezeichnung_2020.csv")
codierung_sh_2019_EP = pd.read_csv("data/Codierung_SH - EP_Bezeichnung_2019.csv")
codierung_sh_2018_EP = pd.read_csv("data/Codierung_SH - EP_Bezeichnung_2018.csv")
codierung_sh_2022_KNr = pd.read_csv("data/Codierung_SH - KapitelNr_Bezeichnung_2022.csv")
codierung_sh_2021_KNr = pd.read_csv("data/Codierung_SH - KapitelNr_Bezeichnung_2021.csv")
codierung_sh_2020_KNr = pd.read_csv("data/Codierung_SH - KapitelNr_Bezeichnung_2020.csv")
codierung_sh_2019_KNr = pd.read_csv("data/Codierung_SH - KapitelNr_Bezeichnung_2019.csv")
codierung_sh_2018_KNr = pd.read_csv("data/Codierung_SH - KapitelNr_Bezeichnung_2018.csv")

Nun müssen die jeweiligen df aufbereitet werden. Wir fangen mit den EP + Bezeichnungen an. Int muss zu str + leading 0 + Columnname anpassen.

In [ ]:
codierung_sh_2022_EP['ep_nr'] = codierung_sh_2022_EP['ep_nr'].astype(str)
codierung_sh_2022_EP['ep_nr'] = codierung_sh_2022_EP['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2022_EP.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2022_EP.insert(len(codierung_sh_2022_EP.columns), "state_id", 15)

codierung_sh_2021_EP['ep_nr'] = codierung_sh_2021_EP['ep_nr'].astype(str)
codierung_sh_2021_EP['ep_nr'] = codierung_sh_2021_EP['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2021_EP.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2021_EP.insert(len(codierung_sh_2021_EP.columns), "state_id", 15)

codierung_sh_2020_EP['ep_nr'] = codierung_sh_2020_EP['ep_nr'].astype(str)
codierung_sh_2020_EP['ep_nr'] = codierung_sh_2020_EP['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2020_EP.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2020_EP.insert(len(codierung_sh_2020_EP.columns), "state_id", 15)

codierung_sh_2019_EP['ep_nr'] = codierung_sh_2019_EP['ep_nr'].astype(str)
codierung_sh_2019_EP['ep_nr'] = codierung_sh_2019_EP['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2019_EP.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2019_EP.insert(len(codierung_sh_2019_EP.columns), "state_id", 15)

codierung_sh_2018_EP['ep_nr'] = codierung_sh_2018_EP['ep_nr'].astype(str)
codierung_sh_2018_EP['ep_nr'] = codierung_sh_2018_EP['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2018_EP.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2018_EP.insert(len(codierung_sh_2018_EP.columns), "state_id", 15)


In [ ]:
codierung_sh_2022_EP

Nun kümmern wir uns um die Kapitel. Wieder int zu str, leading 0 und namen anpassen.

In [ ]:
codierung_sh_2022_KNr['ep_nr'] = codierung_sh_2022_KNr['ep_nr'].astype(str)
codierung_sh_2022_KNr['kapitel_nr'] = codierung_sh_2022_KNr['kapitel_nr'].astype(str)
codierung_sh_2022_KNr['ep_nr'] = codierung_sh_2022_KNr['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2022_KNr['kapitel_nr'] = codierung_sh_2022_KNr['kapitel_nr'].apply(lambda x:x.zfill(3))
codierung_sh_2022_KNr.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2022_KNr.rename(columns={'kapitel_nr':'kapitel'}, inplace=True)
codierung_sh_2022_KNr.insert(len(codierung_sh_2022_KNr.columns), "state_id", 15)

codierung_sh_2021_KNr['ep_nr'] = codierung_sh_2021_KNr['ep_nr'].astype(str)
codierung_sh_2021_KNr['kapitel_nr'] = codierung_sh_2021_KNr['kapitel_nr'].astype(str)
codierung_sh_2021_KNr['ep_nr'] = codierung_sh_2021_KNr['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2021_KNr['kapitel_nr'] = codierung_sh_2021_KNr['kapitel_nr'].apply(lambda x:x.zfill(3))
codierung_sh_2021_KNr.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2021_KNr.rename(columns={'kapitel_nr':'kapitel'}, inplace=True)
codierung_sh_2021_KNr.insert(len(codierung_sh_2021_KNr.columns), "state_id", 15)

codierung_sh_2020_KNr['ep_nr'] = codierung_sh_2020_KNr['ep_nr'].astype(str)
codierung_sh_2020_KNr['kapitel_nr'] = codierung_sh_2020_KNr['kapitel_nr'].astype(str)
codierung_sh_2020_KNr['ep_nr'] = codierung_sh_2020_KNr['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2020_KNr['kapitel_nr'] = codierung_sh_2020_KNr['kapitel_nr'].apply(lambda x:x.zfill(3))
codierung_sh_2020_KNr.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2020_KNr.rename(columns={'kapitel_nr':'kapitel'}, inplace=True)
codierung_sh_2020_KNr.insert(len(codierung_sh_2020_KNr.columns), "state_id", 15)

codierung_sh_2019_KNr['ep_nr'] = codierung_sh_2019_KNr['ep_nr'].astype(str)
codierung_sh_2019_KNr['kapitel_nr'] = codierung_sh_2019_KNr['kapitel_nr'].astype(str)
codierung_sh_2019_KNr['ep_nr'] = codierung_sh_2019_KNr['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2019_KNr['kapitel_nr'] = codierung_sh_2019_KNr['kapitel_nr'].apply(lambda x:x.zfill(3))
codierung_sh_2019_KNr.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2019_KNr.rename(columns={'kapitel_nr':'kapitel'}, inplace=True)
codierung_sh_2019_KNr.insert(len(codierung_sh_2019_KNr.columns), "state_id", 15)

codierung_sh_2018_KNr['ep_nr'] = codierung_sh_2018_KNr['ep_nr'].astype(str)
codierung_sh_2018_KNr['kapitel_nr'] = codierung_sh_2018_KNr['kapitel_nr'].astype(str)
codierung_sh_2018_KNr['ep_nr'] = codierung_sh_2018_KNr['ep_nr'].apply(lambda x:x.zfill(2))
codierung_sh_2018_KNr['kapitel_nr'] = codierung_sh_2018_KNr['kapitel_nr'].apply(lambda x:x.zfill(3))
codierung_sh_2018_KNr.rename(columns={"ep_nr":"ep"}, inplace=True)
codierung_sh_2018_KNr.rename(columns={'kapitel_nr':'kapitel'}, inplace=True)
codierung_sh_2018_KNr.insert(len(codierung_sh_2018_KNr.columns), "state_id", 15)

In [ ]:
codierung_sh_2018_KNr

funktion verändert, um in sql einfügen zu können. Zu dem zweck oben ausge#. mal sehen, ob es klappt.

In [ ]:
def push_to_database(df, table_name, engine, schema):
    if engine!=None:
        try:
            df.to_sql(name=table_name, # Name of SQL table
                            con=engine, # Engine or connection
                            if_exists='append', # Drop the table before inserting new values
                            schema=schema, # Use schmea that was defined earlier
                            index=False, # Write DataFrame index as a column
                            chunksize=5000, # Specify the number of rows in each batch to be written at a time
                            method='multi') # Pass multiple values in a single INSERT clause
            print(f"The {table_name} table was imported successfully.")
        # Error handling
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
            engine = None

In [ ]:
#push_to_database(codierung_sh_2018_EP,"codierung_sh_EP", engine, "capstone_public_budgeting")
#push_to_database(codierung_sh_2018_KNr,"codierung_sh_KNr", engine, "capstone_public_budgeting")